# 评估模型

上面的测试过程是一张一张图片进行预测的，这样的话速度会很慢，所以我们可以对图片进行批量处理，这样的话速度会快很多。所以我们需要对预测函数进行修改，使其能够批量处理图片。

### 定义测试集

In [1]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((256, 256))
])

# 将灰度图的数据集转换成RGB图像
transform_gray = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Resize((256, 256))
])

class TestDataset(torch.utils.data.Dataset):
    def __init__(
            self, 
            data_dir, 
            img_paths,
            transform=transform,
            transform_gray=transform_gray
        ):
        self.data_dir = data_dir
        self.transform = transform
        self.transform_gray = transform_gray
        # self.imgs = os.listdir(data_dir)
        self.imgs = img_paths

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = os.path.join(self.data_dir, self.imgs[idx])
        img = Image.open(img_path)
        if img.mode == 'RGB':
            img = self.transform(img)
        elif img.mode == 'L':
            img = self.transform_gray(img)
        return img

### 查看提交格式

查看提交格式先提前保存好测试集的信息

In [2]:
import pandas as pd
submit_examples = pd.read_csv('starting_kit/test.csv', header = 0)
submit_examples.head()

,image_id,predictions
0,45331,{265.362 70.297 335.99100000000004 165.1499999...
1,35639,{90.562 96.953 138.94299999999998 162.86599999...
2,33770,{407.708 119.523 446.774 166.555 0.58494 6}{41...
3,42479,{472.504 137.652 505.01300000000003 183.505 0....
4,44556,{447.187 46.098 521.98 83.03 0.65175 2}{453.01...


In [3]:
my_submit = pd.DataFrame(columns=['image_id', 'predictions'])
my_submit['image_id'] = submit_examples['image_id']

In [4]:
# 将submit_examples中的image_id转换成文件路径列表，具体形式为'%07d.jpg'%show_img_id
submit_examples['image_path'] = submit_examples['image_id'].apply(lambda x: '%07d.jpg'%x)
submit_examples.head()

,image_id,predictions,image_path
0,45331,{265.362 70.297 335.99100000000004 165.1499999...,0045331.jpg
1,35639,{90.562 96.953 138.94299999999998 162.86599999...,0035639.jpg
2,33770,{407.708 119.523 446.774 166.555 0.58494 6}{41...,0033770.jpg
3,42479,{472.504 137.652 505.01300000000003 183.505 0....,0042479.jpg
4,44556,{447.187 46.098 521.98 83.03 0.65175 2}{453.01...,0044556.jpg


In [5]:
img_paths = submit_examples['image_path'].tolist()
img_paths[:5]

['0045331.jpg', '0035639.jpg', '0033770.jpg', '0042479.jpg', '0044556.jpg']

In [15]:
# 先建立一个csv文件储存测试集图片信息
test_images = pd.DataFrame(columns=['image_id', 'image_path', 'width', 'height'])
test_images['image_id'] = submit_examples['image_id']
test_images['image_path'] = submit_examples['image_path']
for i in range(len(submit_examples)):
    img = Image.open('./dl_detection/test/%07d.jpg'%submit_examples['image_id'][i])
    test_images['width'][i] = img.size[0]
    test_images['height'][i] = img.size[1]

C:\Users\dongl\AppData\Local\Temp/ipykernel_19048/2862501199.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_images['width'][i] = img.size[0]
C:\Users\dongl\AppData\Local\Temp/ipykernel_19048/2862501199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_images['height'][i] = img.size[1]


In [17]:
test_images.to_csv('test_images.csv', index=False)

In [6]:
test_data_dir = "./dl_detection/test/"
Cocodata_test = TestDataset(
    test_data_dir,
    img_paths
)
Cocodata_test.imgs[0], Cocodata_test.__len__()

('0045331.jpg', 16362)

## 开始批量预测

In [7]:
from utils.device import try_gpu
from mydet.model import R50_FPN_SSD
import torch
device = try_gpu()
net = torch.load('checkpoint/resnet50_fpn_ssd.pth',map_location=device)

In [8]:
test_dataloader = torch.utils.data.DataLoader(
    Cocodata_test,
    batch_size=4,
    shuffle=False
)

In [9]:
from mydet.bbox.postprocessor import BBoxPostProcessor
from mydet.bbox.postprocessor import bbox_filter
import torch.nn.functional as F
from utils.submit import bboxListToStr

bbpp = BBoxPostProcessor(
    nms_threshold=0.5,
    neg_threshold=0.00999
)

def predict_batch(X, batch_idx):
    net.eval()
    with torch.no_grad():
        batch_size = X.shape[0]
        anchors, cls_preds, bbox_preds = net(X.to(device))
        cls_probs = F.softmax(cls_preds, dim=2).permute(0, 2, 1) # 利用softmax转换成概率
        output = bbpp.multibox_detection(cls_probs, bbox_preds, anchors)
        # 去除掉背景类
        for sample_idx in range(batch_size):
            # 获取图片的宽和高信息
            image_idx = batch_idx * batch_size + sample_idx
            w = test_images['width'][image_idx]
            h = test_images['height'][image_idx]
            bbox_scale = torch.tensor([w, h, w, h], dtype=torch.float32, device=device)
            # 去除掉背景类
            idx = [i for i, sample in enumerate(output[sample_idx]) if sample[0] != -1]
            sample_without_background = output[sample_idx][idx]
            # 过滤掉一些预测框
            sample_good = bbox_filter(sample_without_background)
            # 如果一个类别的预测框超过300个，只取前300个（置信度从高到低排序）
            if len(sample_good) > 300:
                sample_good = sample_good[:300]

            bbox_xyxy = sample_good[:, 2:] * bbox_scale
            bbox_labels = sample_good[:, 0]
            bbox_confs = sample_good[:, 1]

            sample_good = torch.cat(
                [
                    bbox_xyxy,
                    bbox_confs.unsqueeze(1),
                    bbox_labels.unsqueeze(1),
                ],
                dim=1
            )

            # 将预测框转换成字符串并保存到my_submit中
            my_submit.loc[image_idx, 'predictions'] = bboxListToStr(sample_good.cpu().numpy().tolist())
            print('batch_idx: %d, sample_idx: %d, sample_good: %d' % (batch_idx, sample_idx, len(sample_good)))

    

In [10]:
# 预测
for batch_idx, X in enumerate(test_dataloader):
    predict_batch(X, batch_idx)

d:\anaconda\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


batch_idx: 0, sample_idx: 0, sample_good: 164
batch_idx: 0, sample_idx: 1, sample_good: 234
batch_idx: 0, sample_idx: 2, sample_good: 109
batch_idx: 0, sample_idx: 3, sample_good: 125
batch_idx: 1, sample_idx: 0, sample_good: 173
batch_idx: 1, sample_idx: 1, sample_good: 267
batch_idx: 1, sample_idx: 2, sample_good: 236
batch_idx: 1, sample_idx: 3, sample_good: 223


KeyboardInterrupt: 

In [11]:
my_submit.head()

,image_id,predictions
0,45331,{0.0 0.9972484707832336 0.0013790568336844444 ...
1,35639,{0.0 0.9999841451644897 0.013586888089776039 0...
2,33770,{0.0 0.9425780177116394 0.0011884903069585562 ...
3,42479,{0.0 0.5056626200675964 0.8542353510856628 0.0...
4,44556,{0.0 0.9996554851531982 0.0037318053655326366 ...


In [12]:
my_submit.to_csv('my_submit.csv', index=False)